### Input

In [1]:
%load_ext autoreload
%matplotlib inline
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from helpers import helpfunction as helper
from joblib import dump, load
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve, auc
import pickle

pd.set_option("display.max_columns", 500)

<IPython.core.display.Javascript object>

In [3]:
%autoreload

<IPython.core.display.Javascript object>

### Constants

In [4]:
x_sce_magic = 1.03
pid_upper_clip = 300
pot_target = 1e20

<IPython.core.display.Javascript object>

In [5]:
max_trk_score = 0.15
min_cluster_frac = 0.6

<IPython.core.display.Javascript object>

In [6]:
x_sce_magic = 1.03
pid_upper_clip = 300
pot_target = 1e20

<IPython.core.display.Javascript object>

In [7]:
max_trk_score = 0.15
min_cluster_frac = 0.6

<IPython.core.display.Javascript object>

### Load Samples

In [8]:
run = 1
input_dir = "./input/13Nov/run{}/".format(run)
output_dir = "./output/run{}/".format(run)
model_dir = "./models/run{}/".format(run)

<IPython.core.display.Javascript object>

In [9]:
exclude_samples = []
sample_info, fields = helper.load_sample_info(input_dir, run, exclude_samples)
pot_scale = pot_target / sample_info["on"]["pot"]

Sample Summary: [name, POT, Scaling, Events, SliceID passing rate]


/home/wouter/Binaries/miniconda3/envs/uproot/lib/python3.7/site-packages/awkward/array/jagged.py:1031: RuntimeWarning: overflow encountered in multiply
  result = getattr(ufunc, method)(*inputs, **kwargs)
/home/wouter/Binaries/miniconda3/envs/uproot/lib/python3.7/site-packages/awkward/array/jagged.py:1031: RuntimeWarning: overflow encountered in add
  result = getattr(ufunc, method)(*inputs, **kwargs)
/home/wouter/Binaries/miniconda3/envs/uproot/lib/python3.7/site-packages/awkward/array/jagged.py:1031: RuntimeWarning: overflow encountered in power
  result = getattr(ufunc, method)(*inputs, **kwargs)
/home/wouter/Binaries/miniconda3/envs/uproot/lib/python3.7/site-packages/awkward/array/jagged.py:1031: RuntimeWarning: invalid value encountered in true_divide
  result = getattr(ufunc, method)(*inputs, **kwargs)


dirt 	2.69e+20 POT	Scaling: 0.14	85626 events	 NeutrinoID: 22.8%
nu 	5.37e+20 POT	Scaling: 0.068	423777 events	 NeutrinoID: 42.9%
nu sample: nueccinc passing Slice ID 	85.14%
off 	0 POT	Scaling: 0.26	377927 events	 NeutrinoID: 14.5%
nue 	5.22e+22 POT	Scaling: 0.0007	87875 events	 NeutrinoID: 79.3%
nue sample: nueccinc passing Slice ID 	83.83%
on 	3.66e+19 POT	Scaling: 1	141550 events	 NeutrinoID: 22.9%
Completed, time passed: 66.4s.


<IPython.core.display.Javascript object>

### Explore Tree

In [10]:
shr_fields = [f for f in fields if f.startswith("shr_") and f.endswith("_v")]
trk_fields = [f for f in fields if f.startswith("trk_") and f.endswith("_v")]
backracked_fields = [f for f in fields if f.startswith("backtracked_")]
mc_fields = [f for f in fields if f.startswith(("true_", "mc_"))]
other_fields = (
    set(fields)
    - set(shr_fields)
    - set(trk_fields)
    - set(backracked_fields)
    - set(mc_fields)
)

<IPython.core.display.Javascript object>

In [11]:
trk_fields

['trk_score_v',
 'trk_bragg_p_v',
 'trk_bragg_mu_v',
 'trk_bragg_mip_v',
 'trk_pida_v',
 'trk_pid_chipr_v',
 'trk_pid_chipi_v',
 'trk_pid_chika_v',
 'trk_pid_chimu_v',
 'trk_bragg_p_u_v',
 'trk_bragg_mu_u_v',
 'trk_bragg_mip_u_v',
 'trk_pida_u_v',
 'trk_pid_chipr_u_v',
 'trk_pid_chipi_u_v',
 'trk_pid_chika_u_v',
 'trk_pid_chimu_u_v',
 'trk_bragg_p_v_v',
 'trk_bragg_mu_v_v',
 'trk_bragg_mip_v_v',
 'trk_pida_v_v',
 'trk_pid_chipr_v_v',
 'trk_pid_chipi_v_v',
 'trk_pid_chika_v_v',
 'trk_pid_chimu_v_v',
 'trk_pfp_id_v',
 'trk_dir_x_v',
 'trk_dir_y_v',
 'trk_dir_z_v',
 'trk_start_x_v',
 'trk_start_y_v',
 'trk_start_z_v',
 'trk_sce_start_x_v',
 'trk_sce_start_y_v',
 'trk_sce_start_z_v',
 'trk_end_x_v',
 'trk_end_y_v',
 'trk_end_z_v',
 'trk_sce_end_x_v',
 'trk_sce_end_y_v',
 'trk_sce_end_z_v',
 'trk_distance_v',
 'trk_theta_v',
 'trk_phi_v',
 'trk_len_v',
 'trk_mcs_muon_mom_v',
 'trk_energy_proton_v',
 'trk_energy_muon_v',
 'trk_calo_energy_u_v',
 'trk_calo_energy_v_v',
 'trk_calo_energy_y_v',

<IPython.core.display.Javascript object>

In [12]:
shr_fields

['shr_dedx_u_v',
 'shr_dedx_v_v',
 'shr_dedx_y_v',
 'shr_energy_u_v',
 'shr_energy_v_v',
 'shr_energy_y_v',
 'shr_pfp_id_v',
 'shr_start_x_v',
 'shr_start_y_v',
 'shr_start_z_v',
 'shr_start_U_v',
 'shr_start_V_v',
 'shr_dist_v',
 'shr_nclus0_v',
 'shr_clushitfrac0_v',
 'shr_nclus1_v',
 'shr_clushitfrac1_v',
 'shr_nclus2_v',
 'shr_clushitfrac2_v',
 'shr_px_v',
 'shr_py_v',
 'shr_pz_v',
 'shr_openangle_v',
 'shr_theta_v',
 'shr_phi_v',
 'shr_pitch_u_v',
 'shr_pitch_v_v',
 'shr_pitch_y_v',
 'shr_tkfit_nhits_v',
 'shr_tkfit_start_x_v',
 'shr_tkfit_start_y_v',
 'shr_tkfit_start_z_v',
 'shr_tkfit_start_U_v',
 'shr_tkfit_start_V_v',
 'shr_tkfit_theta_v',
 'shr_tkfit_phi_v',
 'shr_tkfit_pitch_u_v',
 'shr_tkfit_pitch_v_v',
 'shr_tkfit_pitch_y_v',
 'shr_tkfit_dedx_u_v',
 'shr_tkfit_dedx_v_v',
 'shr_tkfit_dedx_y_v',
 'shr_tkfit_gap10_dedx_u_v',
 'shr_tkfit_gap10_dedx_v_v',
 'shr_tkfit_gap10_dedx_y_v',
 'shr_tkfit_dedx_nhits_u_v',
 'shr_tkfit_dedx_nhits_v_v',
 'shr_tkfit_dedx_nhits_y_v',
 'shr_sp

<IPython.core.display.Javascript object>

In [13]:
backracked_fields

['backtracked_pdg',
 'backtracked_e',
 'backtracked_purity',
 'backtracked_completeness',
 'backtracked_overlay_purity',
 'backtracked_px',
 'backtracked_py',
 'backtracked_pz',
 'backtracked_start_x',
 'backtracked_start_y',
 'backtracked_start_z',
 'backtracked_start_t',
 'backtracked_start_U',
 'backtracked_start_V',
 'backtracked_start_Y',
 'backtracked_sce_start_x',
 'backtracked_sce_start_y',
 'backtracked_sce_start_z',
 'backtracked_sce_start_U',
 'backtracked_sce_start_V',
 'backtracked_sce_start_Y']

<IPython.core.display.Javascript object>

In [14]:
other_fields

{'CosmicDirAll2DEnds',
 'CosmicDirAll2DOvlp',
 'CosmicDirAll3D',
 'CosmicIP',
 'CosmicIPAll2DEnds',
 'CosmicIPAll2DOvlp',
 'CosmicIPAll3D',
 'NeutrinoEnergy0',
 'NeutrinoEnergy1',
 'NeutrinoEnergy2',
 'SliceCaloEnergy0',
 'SliceCaloEnergy1',
 'SliceCaloEnergy2',
 'bdt_cosmic',
 'bdt_ext',
 'bdt_global',
 'bdt_nuNCpi0',
 'bdt_numuCC',
 'bdt_numuCCpi0',
 'best_cosmic_flashmatch_score',
 'best_obviouscosmic_flashmatch_score',
 'category',
 'ccnc',
 'cosmic_flashmatch_score_v',
 'crthitpe',
 'crtveto',
 'dmc_boundary',
 'dshr_boundary',
 'dshr_x_boundary',
 'dshr_y_boundary',
 'dshr_z_boundary',
 'dtrk',
 'dtrk_boundary',
 'dtrk_x_boundary',
 'dtrk_y_boundary',
 'dtrk_z_boundary',
 'dvtx',
 'dvtx_boundary',
 'dvtx_x_boundary',
 'dvtx_y_boundary',
 'dvtx_z_boundary',
 'elec_c',
 'elec_dist',
 'elec_e',
 'elec_edep',
 'elec_etot',
 'elec_p',
 'elec_parent',
 'elec_vx',
 'elec_vy',
 'elec_vz',
 'endmuonmichel',
 'endmuonprocess',
 'evgamnhits',
 'evlepnhits',
 'evneunhits',
 'evnhits',
 'evnu

<IPython.core.display.Javascript object>

In [15]:
# search through fields:
[f for f in fields if "ll" in f]

['CosmicIPAll3D',
 'CosmicDirAll3D',
 'CosmicIPAll2DEnds',
 'CosmicDirAll2DEnds',
 'CosmicIPAll2DOvlp',
 'CosmicDirAll2DOvlp',
 'sllepnhits',
 'trk_llr_pid_u_v',
 'trk_llr_pid_v_v',
 'trk_llr_pid_y_v',
 'trk_llr_pid_v',
 'trk_llr_pid_score_v']

<IPython.core.display.Javascript object>

### Add fields

In [16]:
%%time
# For all samples
for k, v in sample_info.items():
    # Add fiducial reco sce vtx
    v["daughters"]["reco_fid_vol"] = np.repeat(
        helper.is_fid(
            *v["daughters"][
                ["reco_nu_vtx_sce_x", "reco_nu_vtx_sce_y", "reco_nu_vtx_sce_z"]
            ]
            .xs(0, level="daughter")
            .values.T
        ),
        v["daughters"]["n_pfps"].xs(0, level="daughter"),
    )
    # Add pfp at vtx:
    v["daughters"]["n_pfpvtx"] = v["daughters"].eval("trk_distance_v<3 & trk_distance_v>=0")
    v["daughters"]["n_pfpvtx"] = v["daughters"]['n_pfpvtx'].groupby("event").transform(sum)
    v["daughters"]["n_pfp_farvtx"] = v["daughters"].eval("n_pfps-n_pfpvtx")
    
    # Add electron candidate check
    e_cand_str = 'pfnplanehits_U>0 & pfnplanehits_V>0 & pfnplanehits_Y>0 & trk_score_v<@max_trk_score & shr_tkfit_dedx_nhits_y_v>0'    
    data = v["daughters"][
                [
                    "pfnplanehits_U", "pfnplanehits_V", "pfnplanehits_Y",
                    "trk_score_v",'shr_tkfit_dedx_nhits_y_v',
                    "shr_dist_v",
                ]
            ]
    electron_candidate = data.eval(e_cand_str)
    e_cand_maxe = data[electron_candidate]["shr_dist_v"].groupby("event").transform(min) == data[electron_candidate]["shr_dist_v"]
    v["daughters"]["e_candidate"] = False
    v["daughters"].loc[e_cand_maxe[e_cand_maxe == True].index, "e_candidate"] = True
    # Add weighted dedx:
    dedx_cols = ["shr_tkfit_dedx_u_v",
                    "shr_tkfit_dedx_v_v",
                    "shr_tkfit_dedx_y_v",
                    'shr_tkfit_nhits_v',
                    "shr_tkfit_dedx_nhits_u_v",
                    "shr_tkfit_dedx_nhits_v_v",
                    "shr_tkfit_dedx_nhits_y_v"]
    v["daughters"][dedx_cols]=v["daughters"][dedx_cols].clip(lower=0)
    str_dedx_weighted_mean='(shr_tkfit_dedx_u_v*shr_tkfit_dedx_nhits_u_v+shr_tkfit_dedx_v_v*shr_tkfit_dedx_nhits_v_v+shr_tkfit_dedx_y_v*shr_tkfit_dedx_nhits_y_v)/(shr_tkfit_dedx_nhits_u_v+shr_tkfit_dedx_nhits_v_v+shr_tkfit_dedx_nhits_y_v)'
    v["daughters"]['shr_tkfit_dedx_wm_v']= v["daughters"].eval(str_dedx_weighted_mean)
    # Add weigthed pid chi mu/pr:
    pid_chi_cols = ["trk_pid_chipr_v",
    "trk_pid_chimu_v",
    "trk_pid_chipr_v_v",
    "trk_pid_chimu_v_v",
    "trk_pid_chipr_u_v",
    "trk_pid_chimu_u_v"]
    v["daughters"][pid_chi_cols]=v["daughters"][pid_chi_cols].clip(upper=pid_upper_clip)
    str_pidmu_weighted_mean='(trk_pid_chimu_v*pfnplanehits_Y+trk_pid_chimu_v_v*pfnplanehits_V+trk_pid_chimu_u_v*pfnplanehits_U)/pfnhits'
    v["daughters"]['trk_pid_chimu_wm_v']= v["daughters"].eval(str_pidmu_weighted_mean)
    str_pidpr_weighted_mean='(trk_pid_chipr_v*pfnplanehits_Y+trk_pid_chipr_v_v*pfnplanehits_V+trk_pid_chipr_u_v*pfnplanehits_U)/pfnhits'
    v["daughters"]['trk_pid_chipr_wm_v']= v["daughters"].eval(str_pidpr_weighted_mean)
    # Add the number of hits per length:
    v["daughters"]['hits_per_tklen']= v["daughters"].eval('pfnhits/trk_len_v')
    
    # Drop columns I do not need anymore:
    drop_cols = [
    "pfnplanehits_U",
    "pfnplanehits_V",
    "pfnplanehits_Y",
    'pfnhits',
    "shr_tkfit_dedx_u_v",
    "shr_tkfit_dedx_v_v",
    "shr_tkfit_nhits_v",
    "shr_tkfit_dedx_nhits_u_v",
    "shr_tkfit_dedx_nhits_v_v",    
    "trk_pid_chipr_v_v",
    "trk_pid_chimu_v_v",
    "trk_pid_chipr_u_v",
    "trk_pid_chimu_u_v",
]
for c in drop_cols:
    if c in v["daughters"].columns:
        v["daughters"].drop(c, inplace=True, axis=1)
    else:
        print(c, " was not in dataframe!")

CPU times: user 2.62 s, sys: 108 ms, total: 2.73 s
Wall time: 1.6 s


<IPython.core.display.Javascript object>

In [18]:
%%time
# For MC samples
for k, v in sample_info.items():
    if k not in helper.data_samples:
        
        # Correct the LEE weight using the splineweight
        v['daughters']["leeweight"]*=v['daughters']["weightSpline"]
        
        # Add distance between reco_sce and true vertex
        data = (
            v["daughters"][
                [
                    "reco_nu_vtx_sce_x",
                    "reco_nu_vtx_sce_y",
                    "reco_nu_vtx_sce_z",
                    "true_nu_vtx_x",
                    "true_nu_vtx_y",
                    "true_nu_vtx_z",
                ]
            ]
            .xs(0, level="daughter")
            .values.T
        )
        data[0]-=x_sce_magic # Correct x location for the 0.6 sign error
        
        v["daughters"]["true_vtx_distance"] = np.repeat(
            np.linalg.norm(data[0:3] - data[3:6], axis=0),
            v["daughters"]["n_pfps"].xs(0, level="daughter"),
        )
        # Cross-check vtx distance
        data = (
            v["daughters"][
                [
                    "true_nu_vtx_sce_x",
                    "true_nu_vtx_sce_y",
                    "true_nu_vtx_sce_z",
                    "reco_nu_vtx_x",
                    "reco_nu_vtx_y",
                    "reco_nu_vtx_z",
                ]
            ]
            .xs(0, level="daughter")
            .values.T
        )
        v["daughters"]["true_vtx_distance_check"] = np.repeat(
            np.linalg.norm(data[0:3] - data[3:6], axis=0),
            v["daughters"]["n_pfps"].xs(0, level="daughter"),
        )
        # Add the distance between the true neutrino vertex and the reconstructed shower start point
        data = (
            v["daughters"][
                [
                    "true_nu_vtx_sce_x",
                    "true_nu_vtx_sce_y",
                    "true_nu_vtx_sce_z",
                    "shr_tkfit_start_x_v",
                    "shr_tkfit_start_y_v",
                    "shr_tkfit_start_z_v",
                ]
            ]
            .values.T
        )
        v["daughters"]["true_shower_distance"] = np.linalg.norm(data[0:3] - data[3:6], axis=0)
        
        # Add the modified purity/completeness to account for overlay.
        overlay_mask = v["daughters"].eval('backtracked_overlay_purity>backtracked_purity')
        v["daughters"].loc[overlay_mask, 'backtracked_pdg'] = 0
        v["daughters"].loc[overlay_mask, 'backtracked_purity'] = v["daughters"].loc[overlay_mask, 'backtracked_overlay_purity']
        v["daughters"].loc[overlay_mask, 'backtracked_completeness'] = 0
        
        # Drop some stuff you do not need anymore
        drop_cols = ['backtracked_overlay_purity',
                     "true_nu_vtx_sce_y",
                     "true_nu_vtx_sce_z",
                     "reco_nu_vtx_y",
                     "reco_nu_vtx_z"]
        for c in drop_cols:
            if c in v["daughters"].columns:
                v["daughters"].drop(c, inplace=True, axis=1)
            else:
                print(c, " was not in dataframe!")

/home/wouter/Binaries/miniconda3/envs/uproot/lib/python3.7/site-packages/numpy/linalg/linalg.py:2512: RuntimeWarning: overflow encountered in multiply
  s = (x.conj() * x).real


CPU times: user 1.35 s, sys: 424 ms, total: 1.77 s
Wall time: 970 ms


<IPython.core.display.Javascript object>

In [ ]:
%%time
# For MC samples, fields needed as training labels:
for k, v in sample_info.items():
    if k in helper.mc_samples:
        # Add training labels and weights
        e_cand = v["daughters"]['e_candidate']
        e_good = v["daughters"].eval('e_candidate & abs(backtracked_pdg)==11 & backtracked_purity>0.75 & backtracked_completeness>0.5')
        e_cand_bad = v["daughters"].eval('e_candidate & abs(backtracked_pdg)!=11')
        other_bad = v["daughters"].eval('~e_candidate & (abs(backtracked_pdg)==13 | backtracked_pdg==22 | backtracked_pdg==0)')
        
        v["daughters"]['train_weight'] = v["daughters"].eval('weightSpline+leeweight*@lee_focus') # weight low energy electrons a bit higher
        v["daughters"]['train_weight'] = v["daughters"].eval('train_weight*(1+(150<shr_energy_y_v<400)*@lee_focus)') # weight low energy electrons a bit higher
        
        v["daughters"].loc[e_good, "train_weight"] *= 5 # can be tuned
        
        v["daughters"]['train_label'] = True
        v["daughters"].loc[e_cand_bad, "train_label"] = False
        v["daughters"].loc[other_bad, "train_label"] = False

In [ ]:
# In case we train on external data:
sample_info["Off"]["daughters"]["nueccinc"] = False
sample_info["Off"]["daughters"]["train_weight"] = 1
low_e_shr = sample_info["Off"]["daughters"].eval("shr_energy_y_v<0.4")
sample_info["Off"]["daughters"].loc[low_e_shr, "train_weight"] *= 5  # can be tuned
sample_info["Off"]["daughters"]["train_label"] = False

### Quality cuts / Pre-selection

In [ ]:
query_preselect = (
    "e_candidate & reco_fid_vol & slclustfrac>@min_cluster_frac & ~crtveto"
)

In [ ]:
%%time
# Passing rate pre-selection
for k, v in sample_info.items():
    v["daughters"]["preselect"] = v["daughters"].eval(
        query_preselect
    )
    v["daughters"]["preselect"] = v["daughters"]["preselect"].groupby("event").transform(max)
    
    pass_rate = sum(v["daughters"].eval("e_candidate & preselect"))/v['numentries']
    print(k, "\t{:.2f}%".format(pass_rate * 100))
    
# Passing rate signal, weights applied!
pass_rate = sum(sample_info["NUE"]['daughters'].eval('(e_candidate & preselect)*weightSpline*nueccinc'))/sum(sample_info["NUE"]["mc"]['weightSpline']*sample_info['NUE']['nueccinc'] )
print("Nue signal passing the preselection \t{:.2f}%".format(pass_rate * 100))
nue_pass = sum(sample_info["NUE"]['daughters'].eval('(e_candidate & preselect)*nueccinc'))*sample_info["NUE"]['scaling']*pot_scale
print("Nue Intrinsic signal passing: {0:0.3f} per {1:0.2g} POT".format(nue_pass,pot_target))
lee_pass = sum(sample_info["NUE"]['daughters'].eval('(e_candidate & preselect)*leeweight*nueccinc'))*sample_info["NUE"]['scaling']*pot_scale
print("Nue LEE signal passing: {0:0.3f} per {1:0.2g} POT".format(lee_pass,pot_target))

## Calculate the purity:
purity_denom = (
    sum(sample_info["MC"]["daughters"].eval("(e_candidate & preselect)*weightSpline"))
    * sample_info["MC"]["scaling"]
)
purity_denom += (
    sum(sample_info["DRT"]["daughters"].eval("(e_candidate & preselect)*weightSpline"))
    * sample_info["DRT"]["scaling"]
)
purity_denom += (
    sum(sample_info["Off"]["daughters"].eval("(e_candidate & preselect)")) * sample_info["Off"]["scaling"]
)

purity_mc = (
    sum(sample_info["NUE"]["daughters"].eval("(e_candidate & preselect)*weightSpline*nueccinc"))
    * sample_info["NUE"]["scaling"]
    / purity_denom
)
purity_data = (
    sum(sample_info["NUE"]["daughters"].eval("(e_candidate & preselect)*weightSpline*nueccinc"))
    * sample_info["NUE"]["scaling"]
    / sum(sample_info["On"]["daughters"].eval("(e_candidate & preselect)"))
)
print(
    "Purity MC: {:.1f}%\nData/MC-ratio: {:.2f}\n".format(
        purity_mc * 100,
        sum(sample_info["On"]["daughters"].eval("(e_candidate & preselect)")) / purity_denom,
    )
)

### Machine Learning Selection

In [ ]:
col_train_electron = [
    # "topological_score",
    # "n_pfpvtx",
    "n_showers",
    # "n_tracks",
    # "trk_score_v",
    # "trk_len_v",
    "trk_pid_chipr_v",
    "trk_pid_chimu_v",
    "trk_min_cos",
    "hits_per_tklen",
    "shr_dist_v",
    "shr_energy_y_v",
    # "shr_openangle_v",
    # "shr_tkfit_theta_v",
    # "shr_tkfit_phi_v",
    "shr_tkfit_dedx_wm_v",
    "shr_tkfit_dedx_y_v",
    # "shr_tkfit_dedx_nhits_y_v",
    ##"train_weight",
    ##"train_label",
]
col_train_other = [
    # "topological_score",
    # "n_showers",
    # "n_tracks",
    # "n_pfpvtx",
    "trk_score_v",
    "trk_distance_v",
    "trk_theta_v",
    # "trk_phi_v",
    "trk_len_v",
    "trk_pid_chimu_wm_v",
    "trk_pid_chipr_wm_v",
    # "trk_min_cos",
    # "hits_per_tklen",
    "shr_energy_y_v",
    # "shr_openangle_v",
    "shr_tkfit_dedx_wm_v",
    ##"train_weight",
    ##"train_label",
]

col_train_event = [
    "topological_score",
    # "n_showers",
    # "n_tracks",
    # "n_pfpvtx",
    "n_pfp_farvtx",
    "hits_ratio",
    "contained_fraction",
    "score",
    # "score_other_max",
    "score_other_min",
    "score_other_mean",
    "CosmicIP"
    # "nu_flashmatch_score",
    ## nueccinc
    ## train_weight -> use the train weight of the electron candidate
]

#### Electron training

In [ ]:
train_q = "preselect & e_candidate"

X_train = []
X_test = []
y_train = []
y_test = []
for k in train_samples:
    X = sample_info[k]["daughters"].query(train_q)[col_train_electron]
    Y = sample_info[k]["daughters"].query(train_q)[["train_label", "train_weight"]]
    X_tr, X_te, y_tr, y_te = train_test_split(
        X, Y, test_size=test_size, random_state=seed
    )
    X_train.append(X_tr)
    X_test.append(X_te)
    y_train.append(y_tr)
    y_test.append(y_te)

# Merge our two samples
X_train = pd.concat(X_train).reset_index(drop=True)
y_train = pd.concat(y_train).reset_index(drop=True)
X_test = pd.concat(X_test).reset_index(drop=True)
y_test = pd.concat(y_test).reset_index(drop=True)

In [ ]:
# fit model no training data
if retrain:
    model_e = XGBClassifier(max_depth=5)
    model_e.fit(X_train, y_train["train_label"], sample_weight=y_train["train_weight"])
    dump(model_e, model_dir + "model_e.pckl")
else:
    model_e = load(model_dir + "model_e.pckl")

In [ ]:
# make predictions for test data
y_pred = model_e.predict(X_test)
target_names = ["electron", "non_electron"]
print(classification_report(y_test["train_label"], y_pred, target_names=target_names))
predictions = [round(value) for value in y_pred]

# evaluate predictions
accuracy = accuracy_score(y_test["train_label"], predictions)
print("Test accuracy: %.2f%%" % (accuracy * 100.0))


# make predictions for test data
y_pred_train = model_e.predict(X_train)
predictions = [round(value) for value in y_pred_train]
# evaluate predictions
accuracy = accuracy_score(y_train["train_label"], predictions)
print("Train accuracy: %.2f%%\n" % (accuracy * 100.0))

importances = model_e.feature_importances_
sort = np.argsort(-importances)

for i, (n, im) in enumerate(zip(np.array(col_train_electron)[sort], importances[sort])):
    print("%d. feature %s (%f)" % (i + 1, n, im))

In [ ]:
fig, ax = plt.subplots(ncols=2, figsize=(8, 4))

y_pred = model_e.predict_proba(X_test).T[1]
fpr, tpr, _ = roc_curve(y_test["train_label"], y_pred)
roc_auc = auc(fpr, tpr)

ax[0].hist(
    y_pred[y_test["train_label"] == 0],
    alpha=0.5,
    bins=50,
    range=(0, 1),
    label="no electrons",
    density=False,
)
ax[0].hist(
    y_pred[(y_test["train_label"] == 1) & (y_test["train_weight"] > 2)],
    alpha=0.5,
    bins=50,
    range=(0, 1),
    label="good electrons",
    density=False,
)
ax[0].hist(
    y_pred[(y_test["train_label"] == 1) & (y_test["train_weight"] < 2)],
    alpha=0.5,
    bins=50,
    range=(0, 1),
    label="bad electrons",
    density=False,
)
ax[0].legend(loc="upper left")
ax[0].set_xlim(0, 1)
ax[0].set_xlabel("Electron identification")
ax[0].set_ylabel("Entries per bin")
ax[0].set_title("Electron Candidate XGB")

ax[1].plot(fpr, tpr, label="ROC curve (area = %0.2f)" % roc_auc)
plt.plot([0, 1], [0, 1], linestyle="--")
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("Receiver Operating Characteristic")
plt.legend(loc="lower right")

fig.tight_layout()
fig.savefig(output_dir + "e_bdt_test.pdf")

#### Other daughters training

In [ ]:
train_q = "preselect & ~e_candidate"
X_train = []
X_test = []
y_train = []
y_test = []
for k in train_samples:
    X = sample_info[k]["daughters"].query(train_q)[col_train_other]
    Y = sample_info[k]["daughters"].query(train_q)[["train_label", "train_weight"]]
    X_tr, X_te, y_tr, y_te = train_test_split(
        X, Y, test_size=test_size, random_state=seed
    )
    X_train.append(X_tr)
    X_test.append(X_te)
    y_train.append(y_tr)
    y_test.append(y_te)

# Merge our two samples
X_train = pd.concat(X_train).reset_index(drop=True)
y_train = pd.concat(y_train).reset_index(drop=True)
X_test = pd.concat(X_test).reset_index(drop=True)
y_test = pd.concat(y_test).reset_index(drop=True)

In [ ]:
# fit model on training data
if retrain:
    model_d = XGBClassifier(max_depth=5)
    model_d.fit(X_train, y_train["train_label"], sample_weight=y_train["train_weight"])
    dump(model_d, model_dir + "model_d.pckl")
else:
    model_d = load(model_dir + "model_d.pckl")

In [ ]:
# make predictions for test data
y_pred = model_d.predict(X_test)
target_names = ["proton, pion, ...", "muon, overlay, photon"]
print(classification_report(y_test["train_label"], y_pred, target_names=target_names))
predictions = [round(value) for value in y_pred]

# evaluate predictions
accuracy = accuracy_score(y_test["train_label"], predictions)
print("Test accuracy: %.2f%%" % (accuracy * 100.0))


# make predictions for test data
y_pred_train = model_d.predict(X_train)
predictions = [round(value) for value in y_pred_train]
# evaluate predictions
accuracy = accuracy_score(y_train["train_label"], predictions)
print("Train accuracy: %.2f%%\n" % (accuracy * 100.0))

importances = model_d.feature_importances_
sort = np.argsort(-importances)

for i, (n, im) in enumerate(zip(np.array(col_train_other)[sort], importances[sort])):
    print("%d. feature %s (%f)" % (i + 1, n, im))

In [ ]:
fig, ax = plt.subplots(ncols=2, figsize=(8, 4))

y_pred = model_d.predict_proba(X_test).T[1]
fpr, tpr, _ = roc_curve(y_test["train_label"], y_pred)
roc_auc = auc(fpr, tpr)

ax[0].hist(
    y_pred[y_test["train_label"] == 0],
    alpha=0.5,
    bins=50,
    range=(0, 1),
    label="muon, overlay, photon",
    density=False,
)
ax[0].hist(
    y_pred[y_test["train_label"] == 1],
    alpha=0.5,
    bins=50,
    range=(0, 1),
    label="proton, pion, ...",
    density=False,
)
ax[0].legend(loc="upper left")
ax[0].set_xlim(0, 1)
ax[0].set_xlabel("Daughter identification")
ax[0].set_ylabel("Entries per bin")
ax[0].set_title("Daughter XGB")

ax[1].plot(fpr, tpr, label="ROC curve (area = %0.2f)" % roc_auc)
plt.plot([0, 1], [0, 1], linestyle="--")
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("Receiver Operating Characteristic")
plt.legend(loc="lower right")

fig.tight_layout()
fig.savefig(output_dir + "daughter_bdt_test.pdf")

#### Apply models on the samples!

In [ ]:
%time
# add the columns for each daughter (all samples)
for k, v in sample_info.items():
    v["daughters"]["score"] = -1
    mask_e_cand = v["daughters"].eval("preselect & e_candidate")
    v["daughters"].loc[mask_e_cand, "score"] = model_e.predict_proba(
        v["daughters"][col_train_electron][mask_e_cand]
    ).T[1]
    mask_d = v["daughters"].eval("preselect & ~e_candidate")
    v["daughters"].loc[mask_d, "score"] = model_d.predict_proba(
        v["daughters"][col_train_other][mask_d]
    ).T[1]

#### Train on the events

In [ ]:
%%time
# First we need to add some columns:
for k, v in sample_info.items():    
    mask_e = v["daughters"].eval("preselect & e_candidate & n_pfps>1")
    v["daughters"]["score_other_max"] = 1
    v["daughters"].loc[mask_e, "score_other_max"] = (
        v["daughters"]
        .query("~e_candidate & preselect")["score"]
        .groupby("event")
        .max()
        .values
    )
    v["daughters"]["score_other_mean"] = 0.95
    v["daughters"].loc[mask_e, "score_other_mean"] = (
        v["daughters"]
        .query("~e_candidate & preselect")["score"]
        .groupby("event")
        .mean()
        .values
    )
    v["daughters"]["score_other_min"] = 0.9
    v["daughters"].loc[mask_e, "score_other_min"] = (
        v["daughters"]
        .query("~e_candidate & preselect")["score"]
        .groupby("event")
        .min()
        .values
    )

In [ ]:
train_q = "preselect & e_candidate"

X_train = []
X_test = []
y_train = []
y_test = []
for k in train_samples:
    X = sample_info[k]["daughters"].query(train_q)[col_train_event]
    Y = sample_info[k]["daughters"].query(train_q)[["nueccinc", "train_weight"]]
    X_tr, X_te, y_tr, y_te = train_test_split(
        X, Y, test_size=test_size, random_state=seed
    )
    X_train.append(X_tr)
    X_test.append(X_te)
    y_train.append(y_tr)
    y_test.append(y_te)

# Merge our two samples
X_train = pd.concat(X_train).reset_index(drop=True)
y_train = pd.concat(y_train).reset_index(drop=True)
X_test = pd.concat(X_test).reset_index(drop=True)
y_test = pd.concat(y_test).reset_index(drop=True)

In [ ]:
# fit model on training data\
if retrain:
    model_event = XGBClassifier(max_depth=4, n_estimators=200)
    model_event.fit(X_train, y_train["nueccinc"], sample_weight=y_train["train_weight"])
    dump(model_event, model_dir + "model_event.pckl")
else:
    model_event = load(model_dir + "model_event.pckl")

In [ ]:
# make predictions for test data
y_pred = model_event.predict(X_test)
target_names = ["nueccinc", "background events"]
print(classification_report(y_test["nueccinc"], y_pred, target_names=target_names))
predictions = [round(value) for value in y_pred]

# evaluate predictions
accuracy = accuracy_score(y_test["nueccinc"], predictions)
print("Test accuracy: %.2f%%" % (accuracy * 100.0))


# make predictions for test data
y_pred_train = model_event.predict(X_train)
predictions = [round(value) for value in y_pred_train]
# evaluate predictions
accuracy = accuracy_score(y_train["nueccinc"], predictions)
print("Train accuracy: %.2f%%\n" % (accuracy * 100.0))

importances = model_event.feature_importances_
sort = np.argsort(-importances)

for i, (n, im) in enumerate(zip(np.array(col_train_event)[sort], importances[sort])):
    print("%d. feature %s (%f)" % (i + 1, n, im))

In [ ]:
fig, ax = plt.subplots(ncols=2, figsize=(8, 4))

y_pred = model_event.predict_proba(X_test).T[1]
fpr, tpr, _ = roc_curve(y_test["nueccinc"], y_pred)
roc_auc = auc(fpr, tpr)

ax[0].hist(
    y_pred[y_test["nueccinc"] == 0],
    alpha=0.5,
    bins=50,
    range=(0, 1),
    label=r"$\nu_e$ CC Inclusive",
    density=False,
)
ax[0].hist(
    y_pred[y_test["nueccinc"] == 1],
    alpha=0.5,
    bins=50,
    range=(0, 1),
    label="background events",
    density=False,
)
ax[0].legend(loc="upper left")
ax[0].set_xlim(0, 1)
ax[0].set_xlabel("Event identification")
ax[0].set_ylabel("Entries per bin")
ax[0].set_title("Event XGB")

ax[1].plot(fpr, tpr, label="ROC curve (area = %0.2f)" % roc_auc)
plt.plot([0, 1], [0, 1], linestyle="--")
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("Receiver Operating Characteristic")
plt.legend(loc="lower right")

fig.tight_layout()
fig.savefig(output_dir + "event_bdt_test.pdf")

In [ ]:
%time
# add the columns for each daughter (all samples)
for k, v in sample_info.items():
    v["daughters"]["score_event"] = -1
    mask_e_cand = v["daughters"].eval("preselect & e_candidate")
    v["daughters"].loc[mask_e_cand, "score_event"] = model_event.predict_proba(
        v["daughters"][col_train_event][mask_e_cand]
    ).T[1]

In [ ]:
query_select = "e_candidate & preselect & score_event>0.9898"
# Passing rate selection
for k, v in sample_info.items():

    v["daughters"]["select"] = v["daughters"].eval(query_select)
    v["daughters"]["select"] = v["daughters"]["select"].groupby("event").transform(max)

    pass_rate = sum(v["daughters"].eval("(e_candidate & select)")) / v["numentries"]
    print(k, "\t{:.4f}%".format(pass_rate * 100))

# Passing rate signal, weights applied!
pass_rate = sum(
    sample_info["NUE"]["daughters"].eval("(e_candidate & select)*weightSpline*nueccinc")
) / sum(sample_info["NUE"]["mc"]["weightSpline"] * sample_info["NUE"]["nueccinc"])
print("Nue signal passing the selection \t{:.2f}%".format(pass_rate * 100))
nue_pass = (
    sum(
        sample_info["NUE"]["daughters"].eval(
            "(e_candidate & select)*weightSpline*nueccinc"
        )
    )
    * sample_info["NUE"]["scaling"]
    * pot_scale
)
print(
    "Nue Intrinsic signal passing: {0:0.3f} per {1:0.2g} POT".format(
        nue_pass, pot_target
    )
)
lee_pass = (
    sum(
        sample_info["NUE"]["daughters"].eval(
            "(e_candidate & select)*leeweight*nueccinc"
        )
    )
    * sample_info["NUE"]["scaling"]
    * pot_scale
)
print("Nue LEE signal passing: {0:0.3f} per {1:0.2g} POT".format(lee_pass, pot_target))

## Calculate the purity:
purity_denom = (
    sum(sample_info["MC"]["daughters"].eval("(e_candidate & select)*weightSpline"))
    * sample_info["MC"]["scaling"]
)
purity_denom += (
    sum(sample_info["DRT"]["daughters"].eval("(e_candidate & select)*weightSpline"))
    * sample_info["DRT"]["scaling"]
)
purity_denom += (
    sum(sample_info["Off"]["daughters"].eval("(e_candidate & select)"))
    * sample_info["Off"]["scaling"]
)

purity_mc = (
    sum(
        sample_info["NUE"]["daughters"].eval(
            "(e_candidate & select)*weightSpline*nueccinc"
        )
    )
    * sample_info["NUE"]["scaling"]
    / purity_denom
)
purity_data = (
    sum(
        sample_info["NUE"]["daughters"].eval(
            "(e_candidate & select)*weightSpline*nueccinc"
        )
    )
    * sample_info["NUE"]["scaling"]
    / sum(sample_info["On"]["daughters"].eval("(e_candidate & select)"))
)
print(
    "Purity MC: {:.1f}%\nData/MC-ratio: {:.2f}\n".format(
        purity_mc * 100,
        sum(sample_info["On"]["daughters"].eval("(e_candidate & select)"))
        / purity_denom,
    )
)

In [ ]:
np.savetxt(
    output_dir + "events_run{}.txt".format(run),
    sample_info["On"]["daughters"].query("(e_candidate & select)")[
        ["run", "sub", "evt"]
    ],
    fmt="%u",
)
sample_info["On"]["daughters"].query("(e_candidate & select)")

In [ ]:
%%time
pickle.dump(sample_info, open(input_dir+"samples.pckl", "wb"))

### Plots

In [ ]:
fig, ax = plt.subplots(ncols=2, figsize=(8, 4))

ax[0].hist(
    sample_info["NUE"]["daughters"]
    .query("nueccinc")["true_vtx_distance"]
    .xs(0, level="daughter"),
    bins=50,
    range=(0, 5),
    label="|true_nu_vtx - reco_nu_vtx_sce|",
    alpha=0.7,
)
ax[0].hist(
    sample_info["NUE"]["daughters"]
    .query("nueccinc")["true_vtx_distance_check"]
    .xs(0, level="daughter"),
    bins=50,
    range=(0, 5),
    label="|true_nu_vtx_sce - reco_nu_vtx|",
    alpha=0.7,
)
ax[0].set_xlabel("Neutrino vertex distance [cm]")
ax[0].set_title(r"$\nu_e$ CC Inclusive", loc="left")
ax[0].set_title("Run3 Overlay", loc="right")
ax[0].legend()

ax[1].hist(
    sample_info["NUE"]["daughters"]
    .query("nueccinc")
    .xs(0, level="daughter")
    .eval("true_nu_vtx_x+@x_sce_magic-reco_nu_vtx_sce_x"),
    bins=51,
    range=(-2, 2),
    label="|true_nu_vtx - reco_nu_vtx_sce|",
    alpha=0.7,
)
ax[1].hist(
    sample_info["NUE"]["daughters"]
    .query("nueccinc")
    .xs(0, level="daughter")
    .eval("true_nu_vtx_sce_x-reco_nu_vtx_x"),
    bins=51,
    range=(-2, 2),
    label="|true_nu_vtx_sce - reco_nu_vtx|",
    alpha=0.7,
)
ax[1].set_xlabel("Neutrino vertex distance x-coordinate [cm]")
ax[1].set_title(r"$\nu_e$ CC Inclusive", loc="left")
ax[1].set_title("Run3 Overlay", loc="right")
ax[1].set_ylim(0, ax[1].get_ylim()[1] * 1.25)
ax[1].legend()

fig.tight_layout()
fig.savefig(output_dir + "vtx_distance.pdf")

In [ ]:
fig, ax = plt.subplots(ncols=1, figsize=(9, 4))

data = sample_info["NUE"]["daughters"].query("e_candidate & abs(backtracked_pdg==11)")[
    ["shr_tkfit_dedx_y_v", "shr_tkfit_dedx_wm_v"]
]

for col in data.columns:
    ax.hist(data[col], bins=50, range=(0, 20), label=col, alpha=0.5)
ax.legend()

fig.tight_layout()
fig.savefig(output_dir + "e_preselection_dedx.pdf")